In [3]:
import pandas as pd
import seaborn as sns
import statsmodels.formula.api as smf
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import math
import datetime as dt

In [4]:
train = pd.read_csv("data/2022_FH.csv")
train = train.dropna(subset = ["Game Prep"])
train.reset_index()

/var/folders/j_/vxxst5h10rs21qx29q6pl7bh0000gn/T/ipykernel_90548/92136705.py:1: DtypeWarning: Columns (59,63,67) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv("data/2022_FH.csv")


,index,Week,Day,W L,Game Prep,Position,Athlete,Session Name,Session,Phase Name,...,TSB HSD,Decel Sum,Acute Decel,Chronic Decel,TSB Decel,HR 4 Percent,HR 5 Percent,Percent Above 80,TRIMP,Duration (min)
0,0,1,Wednesday,W,UConn,B,Maddie Bacskai,GD-2,Training,Whole session,...,1.00,25.0,12.25,12.25,1.0,11.43%,2.59%,14.02%,1222.39,216
1,1,1,Wednesday,W,UConn,F,Bente Baekers,GD-2,Training,Whole session,...,1.00,52.0,25.25,25.25,1.0,10.25%,0.80%,11.06%,1123.52,216
2,2,1,Wednesday,W,UConn,M,Jordan Carr,GD-2,Training,Whole session,...,1.00,74.0,36.50,36.50,1.0,18.93%,1.18%,20.11%,1240.37,216
3,3,1,Wednesday,W,UConn,M,Regan Cornelius,GD-2,Training,Whole session,...,1.00,71.0,34.50,34.50,1.0,9.56%,0.20%,9.76%,1132.51,216
4,4,1,Wednesday,W,UConn,B,Shannon Daley,GD-2,Training,Whole session,...,1.00,87.0,42.75,42.75,1.0,8.42%,0.56%,8.99%,1150.49,216
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16820,22398,65,Sunday,L,NCAA_UNC,M,Chloe Relford,NCAA_UNC,Game,4th quarter,...,1.17,19.0,36.18,32.85,1.1,23.59%,16.36%,39.95%,132.31,22
16821,22399,65,Sunday,L,NCAA_UNC,GK,Annabel Skubisz,NCAA_UNC,Game,4th quarter,...,1.68,1.0,4.28,6.16,0.69,8.83%,0.00%,8.83%,128.66,22
16822,22400,65,Sunday,L,NCAA_UNC,M,Lila Slattery,NCAA_UNC,Game,4th quarter,...,0.87,1.0,19.60,19.29,1.02,0.61%,0.00%,0.61%,103.11,22
16823,22401,65,Sunday,L,NCAA_UNC,M,Lauren Wadas,NCAA_UNC,Game,4th quarter,...,0.88,22.0,27.36,27.92,0.98,24.81%,13.24%,38.05%,135.05,22


**Cleaning data**
- delete all rows that aren't preparing for a game
- only use fall data (August - Dec) - regular season data
- don't include HR% because practices are varying length - actual time is more important
- filter to only include "whole session"
- filter to players who played more than 800 minutes for the 2022 season
- filter out goalies

In [5]:
train = train[["Week", "W L", "Game Prep", "Position", "Athlete", "Session Name", "Phase Name", "Date", "HR Min", "HR AVG", "HR Max", "HR 1", "HR 2", "HR 3", "HR 4", "HR 5", "Total Distance", "Sprints", "Cardio Load", "TLScore", "Duration (min)"]]

In [6]:
train["Date"] = pd.to_datetime(train['Date'])

In [7]:
not_players = ["Lane Herbert", "Katie Jones", "Regan Cornelius", "Jordan Carr", "Greta Hinke", "Lauren Hunter", "Aerin Krys", "Magnolia Powers", "Lila Slattery", "Maja Zivojnovic"]
train = train[(train["Phase Name"] == "Whole session") & (train["Date"] >= "2022-08-01") & (train["Date"] <= "2022-12-01") & (train["Position"] != "GK")]
# not_list = ~train['Athlete'].isin(not_players)
# train = train[not_list]
train.head(30)

,Week,W L,Game Prep,Position,Athlete,Session Name,Phase Name,Date,HR Min,HR AVG,...,HR 1,HR 2,HR 3,HR 4,HR 5,Total Distance,Sprints,Cardio Load,TLScore,Duration (min)
13149,52,NaN,Iowa,M,Amelia Albers,GD-3,Whole session,2022-08-17,90.0,140,...,0:51:58,0:54:02,0:36:55,0:33:36,0:21:30,10895,30,317,247,203
13150,52,NaN,Iowa,F,Bente Baekers,GD-3,Whole session,2022-08-17,83.0,149,...,0:11:58,1:09:52,0:44:16,0:46:51,0:27:57,9668,16,374,291,203
13151,52,NaN,Iowa,M,Jordan Carr,GD-3,Whole session,2022-08-17,45.0,153,...,0:09:55,1:04:56,0:42:49,0:42:52,0:38:39,9310,14,401,346,203
13152,52,NaN,Iowa,M,Regan Cornelius,GD-3,Whole session,2022-08-17,38.0,143,...,0:25:29,1:01:04,0:45:10,0:26:23,0:12:54,12572,84,280,263,203
13153,52,NaN,Iowa,B,Shannon Daley,GD-3,Whole session,2022-08-17,83.0,146,...,0:50:58,0:45:08,0:31:31,0:35:01,0:30:24,10307,43,336,287,203
13154,52,NaN,Iowa,M,Sophie Dix,GD-3,Whole session,2022-08-17,49.0,125,...,0:55:28,0:41:50,0:40:10,0:25:53,0:00:02,12978,75,219,177,203
13155,52,NaN,Iowa,M,Peyton Halsey,GD-3,Whole session,2022-08-17,99.0,166,...,0:00:45,0:06:41,0:14:06,0:09:59,0:16:26,3389,6,132,101,203
13156,52,NaN,Iowa,M,Lane Herbert,GD-3,Whole session,2022-08-17,53.0,133,...,1:09:07,0:54:45,0:29:24,0:37:15,0:01:56,10959,25,251,181,203
13157,52,NaN,Iowa,M,Greta Hinke,GD-3,Whole session,2022-08-17,104.0,154,...,0:10:20,0:57:41,0:50:16,0:47:19,0:35:45,12437,77,402,333,203
13158,52,NaN,Iowa,M,Lauren Hunter,GD-3,Whole session,2022-08-17,18.0,140,...,0:46:17,0:55:38,0:39:59,0:45:55,0:09:16,8574,15,303,218,203
